In [1]:
import pandas as pd

In [2]:
d = {
    'from': ['s','s','s','a','b','c','d','d','d','e','e','f','g'],
    'to': ['a','b','c','d','e','e','f','e','g','g','h','g','h'],
    'length': [10,6,2,7,3,5,5,2,6,4,12,3,5]
}

In [3]:
nds = ['s','a','b','c','d','e','f','g','h']

In [4]:
arcs = pd.DataFrame(data=d)

In [5]:
arcs

,from,to,length
0,s,a,10
1,s,b,6
2,s,c,2
3,a,d,7
4,b,e,3
5,c,e,5
6,d,f,5
7,d,e,2
8,d,g,6
9,e,g,4


In [6]:
import sys

class Vertex:
    def __init__(self, node, m):
        self.id = node
        self.adjacent = {}
        # Set distance to infinity for all nodes
        self.distance = {}
        for mask in range(m**2):
            self.distance[mask] = sys.maxsize
        # Mark all nodes unvisited        
        self.visited = False  
        # Predecessor
        self.previous = {}
        # Bit mask
        self.bit_mask = 0

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]

    def set_distance(self, bm, dist):
        self.distance[bm] = dist

    def get_distance(self, bm = -1):
        if bm == -1:
            return self.distance[self.bit_mask]
        return self.distance[bm]
        
    def set_bit_mask(self, bm):
        self.bit_mask = bm
    
    def get_bit_mask(self):
        return self.bit_mask

    def set_previous(self, bm, prev):
        self.previous[bm] = prev
        
    def get_previous(self, bm):
        if bm in self.previous.keys():
            return self.previous[bm]
        else:
            return None

    def set_visited(self):
        self.visited = True

    def __lt__(self, nxt):
        return self.distance[self.bit_mask] < nxt.distance[self.bit_mask]

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node, m):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node, m)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()

    def set_previous(self, current):
        self.previous = current

    def get_previous(self, current):
        return self.previous

def shortest(v, bm, must_visit_nodes, path):
    ''' make shortest path from v.previous'''
    new_bm = bm
    if v.get_id() in must_visit_nodes:
        new_bm = bm ^ (1 << must_visit_nodes[v.get_id()])
    previous = v.get_previous(new_bm)
    if previous:
        path.append(previous.get_id())
        shortest(previous, new_bm, must_visit_nodes, path)
    return

def get_path(g, target, bm, must_visit_nodes):
    path = [target.get_id()]
    shortest(target, bm, must_visit_nodes, path)
    # Return reversed path
    return path[::-1]

import heapq

def dijkstra(aGraph, start, must_visit_nodes):
    '''Modified dijkstra's shortest path'''
    # Set the distance for the start node to zero 
    start.set_distance(0,0)

    # Put tuple pair into the priority queue
    unvisited_queue = [(v.get_distance(),v) for v in aGraph]
    heapq.heapify(unvisited_queue)

    while len(unvisited_queue):
        # Pops a vertex with the maximum rate 
        uv = heapq.heappop(unvisited_queue)
        current = uv[1]
        current.set_visited()

        #for next in v.adjacent:
        for next in current.adjacent:
            
            new_bit_mask = current.get_bit_mask()
            if next.get_id() in must_visit_nodes:
                new_bit_mask = current.get_bit_mask() | (1 << must_visit_nodes[next.get_id()])
            new_dist = current.get_distance() + current.get_weight(next)
            
            print(new_bit_mask)
            if new_dist < next.get_distance(new_bit_mask):
                next.set_distance(new_bit_mask, new_dist)
                next.set_previous(current.bit_mask, current)
                next.set_bit_mask(new_bit_mask)
                print ('updated : current = %s next = %s new_distance = %s' \
                        %(current.get_id(), next.get_id(), next.get_distance(new_bit_mask)))
            else:
                print ('not updated : current = %s next = %s new_distance = %s' \
                        %(current.get_id(), next.get_id(), next.get_distance(new_bit_mask)))

        # Rebuild heap
        # 1. Pop every item
        while len(unvisited_queue):
            heapq.heappop(unvisited_queue)
        # 2. Put all vertices not visited into the queue
        unvisited_queue = [(v.get_distance(),v) for v in aGraph if not v.visited]
        heapq.heapify(unvisited_queue)


if __name__ == '__main__':

    g = Graph()

In [7]:
must_visit_nodes = {'e':0,'d':1}
# Number of must visted nodes
m = len(must_visit_nodes)

for v in nds:
    g.add_vertex(v, m)

arcs.apply(lambda x: g.add_edge(x['from'], x['to'], x['length']), axis=1)

# Source node
source = g.get_vertex('s')
# Target node
destination = g.get_vertex('g')

# Calling Dijkstra algorithm with source node and must visited nodes
dijkstra(g, source, must_visit_nodes)

0
updated : current = s next = a new_distance = 10
0
updated : current = s next = b new_distance = 6
0
updated : current = s next = c new_distance = 2
0
not updated : current = c next = s new_distance = 0
1
updated : current = c next = e new_distance = 7
0
not updated : current = b next = s new_distance = 0
1
not updated : current = b next = e new_distance = 7
1
updated : current = e next = b new_distance = 10
1
updated : current = e next = c new_distance = 12
3
updated : current = e next = d new_distance = 9
1
updated : current = e next = g new_distance = 11
1
updated : current = e next = h new_distance = 19
3
updated : current = d next = a new_distance = 16
3
updated : current = d next = f new_distance = 14
3
updated : current = d next = e new_distance = 11
3
updated : current = d next = g new_distance = 15
3
not updated : current = f next = d new_distance = 9
3
not updated : current = f next = g new_distance = 15
3
not updated : current = g next = d new_distance = 9
3
not updated : 

In [8]:
bm = (1 << m) - 1
print('Distance: %3d' %destination.get_distance(bm))
print('Path: ',get_path(g, destination, bm, must_visit_nodes))

Distance:  15
Path:  ['s', 'c', 'e', 'd', 'g']


In [9]:
# Best alternative solution with maximum must pass node included 
min_dist = sys.maxsize
target = None
bm = (1 << m)
while target is None:
    bm -= 1
    for n in must_visit_nodes.keys():
        v = g.get_vertex(n)
        d = v.get_distance(bm)
        if d < min_dist:
            min_dist = d
            target = v
            
if target:
    print('Distance: %3d' %min_dist)
    print('Path: ',get_path(g, target, bm, must_visit_nodes))
else:
    print('Not Found')
    

Distance:   9
Path:  ['s', 'c', 'e', 'd']
